# CONVOLUTIONAL NEURONAL NETWORKS AND ARTIFICIAL NEURONAL NETWORKS FOR FLOW PATTERNS CLASSIFICATION

In [ ]:
!nvidia-smi ### GPU

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
## Database address, enter your own Drive address.
PATH = './BDOShohamIML.csv' 
path = "./" #Path for metric images

## Libraries

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
import time as tm

import warnings
warnings.filterwarnings('ignore')

## Dataset

In [ ]:
## Loading Data

# Velocity, Viscosity, Density, Surface Tension, Angle and Diameter
dataset = pd.DataFrame(pd.read_csv(PATH), columns=['Vsl', 'Vsg', 'VisL', 'VisG', 'DenL', 'DenG', 'ST', 'Ang', 'ID', 'Flow Pattern']) 

# Summarize the Dataset 
print("shape of initial data =",dataset.shape) 
# Class Distribution 
print(dataset.groupby('Flow Pattern').size()) 
# Leaving only the best training variables
dataset = dataset.drop(['VisG', 'VisL','DenG', 'ST', 'DenL'], axis=1) #Delete this variables
print("shape of selected data =",dataset.shape) 

print(dataset.head()) 

# Split-out validation dataset 
array = dataset.values 
X = array[:,0:4] #Data or features 
Y = array[:,4]   #Label or classes 
validation_size = 0.20
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=validation_size, random_state=510) 

## Data preparation for neural networks
X_train_ = X_train
X_test_ = X_test

#Renaming labels for y_train
Yt = y_train.copy()
for i in range(len(Yt)):
    if Yt[i] == 'A':
        Yt[i] = 0
    elif Yt[i] == 'B':
        Yt[i] = 1
    elif Yt[i] == 'DB':
        Yt[i] = 2
    elif Yt[i] == 'I':
        Yt[i] = 3
    elif Yt[i] == 'SS':
        Yt[i] = 4
    elif Yt[i] == 'SW':
        Yt[i] = 5
    else:
        print("Dimesion error")

Yt = np.array(np.eye(6)[np.array(Yt, dtype=int)], dtype=np.float32) #For the Softmax activation function

#Renaming labels for y_train
Yv = y_test.copy()
for i in range(len(Yv)):
    if Yv[i] == 'A':
        Yv[i] = 0
    elif Yv[i] == 'B':
        Yv[i] = 1
    elif Yv[i] == 'DB':
        Yv[i] = 2
    elif Yv[i] == 'I':
        Yv[i] = 3
    elif Yv[i] == 'SS':
        Yv[i] = 4
    elif Yv[i] == 'SW':
        Yv[i] = 5
    else:
        print("Dimesion error")

Yv = np.array(np.eye(6)[np.array(Yv, dtype=int)], dtype=np.float32) #For the Softmax activation function

X_train = np.expand_dims(X_train,axis=2)
X_test  = np.expand_dims(X_test,axis=2)
X_train = np.array(X_train, dtype=np.float64)
X_test  = np.array(X_test, dtype=np.float64)
y_train = Yt
y_test = Yv
print("\ntrain data shape =",X_train.shape) 
print("train labels shape =",y_train.shape) 
print("test data shape =",X_test.shape) 
print("test labels shape =",y_test.shape) 


## CONVOLUTIONAL NEURAL NETWORKS

In [ ]:
def modelCNN():
  ip = tf.keras.layers.Input(shape=[X_train_.shape[1],1])
  layers = tf.keras.layers.Conv1D(filters=32, kernel_size=(1), activation='relu')(ip) 
  layers = tf.keras.layers.Conv1D(filters=32, kernel_size=(1), activation='relu')(layers)
  layers = tf.keras.layers.MaxPooling1D(1)(layers)
  layers = tf.keras.layers.Conv1D(filters=32, kernel_size=(1), activation='relu')(layers) 
  layers = tf.keras.layers.Flatten()(layers)
  layers = tf.keras.layers.Dense(512, activation="selu")(layers) 
  layers = tf.keras.layers.Dense(256, activation="selu")(layers)
  layers = tf.keras.layers.Dense(128, activation="selu")(layers)
  layers = tf.keras.layers.Dense(64, activation="selu")(layers) 
  x = tf.keras.layers.Dense(6, activation="softmax", name="output_1")(layers) 
  model = tf.keras.Model(inputs=ip, outputs=x) 
  optimizer = tf.keras.optimizers.RMSprop() 
  model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
  return model 


In [ ]:
modelCNN = modelCNN() 

In [ ]:
# BEST CNN MODEL
epochs = 700
batch_size=64

lossTRAIN,accuracyTRAIN = modelCNN.evaluate(X_train, y_train, verbose=None)
lossTEST,accuracyTEST   = modelCNN.evaluate(X_test, y_test, verbose=None)
start_time = tm.time()
history=modelCNN.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test),verbose=2)
TIME = tm.time() - start_time
print("\nTime: {0:.4f} [seconds]".format(TIME))

with plt.style.context('seaborn-white'):
    plt.figure(figsize=(10, 10))
    #Plot training & validation accuracy values
    plt.plot(np.concatenate([np.array([accuracyTRAIN]),np.array(history.history['accuracy'])],axis=0))
    plt.plot(np.concatenate([np.array([accuracyTEST]), np.array(history.history['val_accuracy'])],axis=0))
    plt.title('Accuracy Vs Epoch')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.grid('on')
    #plt.savefig("./path/CNN_acc.pdf", format='pdf')
    plt.show()
    
    plt.figure(figsize=(10, 10))
    #Plot training & validation loss values
    plt.plot(np.concatenate([np.array([lossTRAIN]),np.array(history.history['loss'])],axis=0))
    plt.plot(np.concatenate([np.array([lossTEST]), np.array(history.history['val_loss'])],axis=0))
    plt.title('Loss Vs Epoch')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.grid('on')
    #plt.savefig("./path/CNN_loss.pdf", format='pdf')
    plt.show()

## ARTIFICIAL NEURAL NETWORKS

In [ ]:
def modelANN():
  ip = tf.keras.layers.Input(shape=[X_train_.shape[1]])
  layers = tf.keras.layers.Dense(512, activation="selu")(ip) 
  layers = tf.keras.layers.Dense(256, activation="selu")(layers)
  layers = tf.keras.layers.Dense(128, activation="selu")(layers)
  layers = tf.keras.layers.Dense(64, activation="selu")(layers) 
  x = tf.keras.layers.Dense(6, activation="softmax", name="output_1")(layers) 
  model = tf.keras.Model(inputs=ip, outputs=x) 
  optimizer = tf.keras.optimizers.RMSprop() 
  model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
  return model 


In [ ]:
modelANN = modelANN() 

In [ ]:
# BEST ANN MODEL
epochs = 1200
batch_size=64

lossTRAIN,accuracyTRAIN = modelANN.evaluate(X_train, y_train, verbose=None)
lossTEST,accuracyTEST   = modelANN.evaluate(X_test, y_test, verbose=None)
start_time = tm.time()
history=modelANN.fit(X_train, y_train, epochs=epochs, batch_size=64, validation_data=(X_test, y_test),verbose=2)
TIME = tm.time() - start_time
print("\nTime: {0:.4f} [seconds]".format(TIME))

with plt.style.context('seaborn-white'):
    plt.figure(figsize=(10, 10))
    #Plot training & validation accuracy values
    plt.plot(np.concatenate([np.array([accuracyTRAIN]),np.array(history.history['accuracy'])],axis=0))
    plt.plot(np.concatenate([np.array([accuracyTEST]), np.array(history.history['val_accuracy'])],axis=0))
    plt.title('Accuracy Vs Epoch')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.grid('on')
    #plt.savefig("./path/ANN_acc.pdf", format='pdf')
    plt.show()
    
    plt.figure(figsize=(10, 10))
    #Plot training & validation loss values
    plt.plot(np.concatenate([np.array([lossTRAIN]),np.array(history.history['loss'])],axis=0))
    plt.plot(np.concatenate([np.array([lossTEST]), np.array(history.history['val_loss'])],axis=0))
    plt.title('Loss Vs Epoch')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.grid('on')
    #plt.savefig("./path/ANN_loss.pdf", format='pdf')
    plt.show()